In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import fashion_mnist

# Load the MNIST data.
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# If subtract pixel mean is enabled
if False:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Input image dimensions.
input_shape = x_train.shape[1:]

The depth of the network can be changed with parameter n.
The fuzzy classifier can be changed with the use_anfis parameter.

In [ ]:
from keras.layers import *
from model import LogGaussMF
from resnet_backend import *


# Prepare model model saving directory.
def get_callbacks(model_type):
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = "%s_model.{epoch:03d}.h5" % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='val_acc',
        verbose=1,
        save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(
        factor=np.sqrt(0.1),
        cooldown=0,
        patience=5,
        min_lr=0.5e-6)

    return [checkpoint, lr_reducer, lr_scheduler]


use_anfis = True

n = 3
depth = n * 9 + 2

# Model name, depth and version
model_type = 'Fuzzy-ResNet%dv%d' % (depth, 2)

inputs, features = resnet_backend_v2(
    input_shape=input_shape, 
    depth=depth)

memberships = LogGaussMF(10)(features)
rules = Lambda(lambda x: K.sum(x, axis=-1))(memberships)

if use_anfis:
    linear = Dense(10)(features)
    logits = Add()([rules, linear])
else:
    logits = rules

outputs = Activation("softmax")(logits)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

callbacks = get_callbacks(model_type)

In [ ]:
history = model.fit(
    x=x_train, y=y_train,
    validation_data=(x_test, y_test),
    epochs=200,
    batch_size=64,
    verbose=1, 
    callbacks=callbacks)

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(12, 20), squeeze=True)
axes[0].set_title("Loss")
axes[0].plot(history.history["loss"], c="b")
axes[0].plot(history.history["val_loss"], c="r")

axes[1].set_title("Log Loss")
axes[1].plot(np.log(history.history["loss"]), c="b")
axes[1].plot(np.log(history.history["val_loss"]), c="r")

axes[2].set_title("Accuracy")
axes[2].plot(history.history["acc"], c="b")
axes[2].plot(history.history["val_acc"], c="r")

axes[3].set_title("Accuracy")
axes[3].set_ylim((0.99, 1.01))
axes[3].plot(history.history["acc"], c="b")
axes[3].plot(history.history["val_acc"], c="r")

axes[4].set_title("Log Learning Rate")
axes[4].plot(np.log(history.history["lr"]), c="b")